# Heart Classification

This notebook demonstrates a complete ML lifecycle:
1.  **Database Normalization**
2.  **Data Loading**
3.  **EDA & Preprocessing**
4.  **Experiment Tracking**
5.  **Model Selection**


## 1. Setup and Imports


In [ ]:
import pandas as pd
import numpy as np
import sqlite3
import os
import joblib
import json
import mlflow
import optuna

from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.base import clone


RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)


DATA_DIR = '../Data'
DB_PATH = os.path.join(DATA_DIR, 'heart.db')
CSV_PATH = os.path.join(DATA_DIR, 'heart.csv')
MODELS_DIR = '../models'

os.makedirs(MODELS_DIR, exist_ok=True)


## 2. Database Creation & Normalization

We will read the raw CSV and convert it into a normalized SQLite schema.

**Schema Design**:
-   `patients`: `id` (generated), `age`, `sex`
-   `lookup_cp`, `lookup_restecg`, `lookup_slope`, `lookup_thal`: Reference tables for categorical codes.
-   `exams`: Foreign keys to patients and lookups, plus remaining measurements.


In [ ]:
def create_normalized_db(csv_path, db_path):
    if os.path.exists(db_path):
        os.remove(db_path)
    
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    
    
    df = pd.read_csv(csv_path)
    
    df['patient_id'] = range(1, len(df) + 1)
    

    cp_values = sorted(df['cp'].unique())
    cursor.execute("CREATE TABLE lookup_cp (cp_code INTEGER PRIMARY KEY, description TEXT)")
    cursor.executemany("INSERT INTO lookup_cp (cp_code, description) VALUES (?, ?)", 
                       [(int(x), f"cp_{x}") for x in cp_values])
                       
    
    restecg_values = sorted(df['restecg'].unique())
    cursor.execute("CREATE TABLE lookup_restecg (restecg_code INTEGER PRIMARY KEY, description TEXT)")
    cursor.executemany("INSERT INTO lookup_restecg (restecg_code, description) VALUES (?, ?)", 
                       [(int(x), f"restecg_{x}") for x in restecg_values])
                       
    
    slope_values = sorted(df['slope'].unique())
    cursor.execute("CREATE TABLE lookup_slope (slope_code INTEGER PRIMARY KEY, description TEXT)")
    cursor.executemany("INSERT INTO lookup_slope (slope_code, description) VALUES (?, ?)", 
                       [(int(x), f"slope_{x}") for x in slope_values])
                       
    
    thal_values = sorted(df['thal'].unique())
    cursor.execute("CREATE TABLE lookup_thal (thal_code INTEGER PRIMARY KEY, description TEXT)")
    cursor.executemany("INSERT INTO lookup_thal (thal_code, description) VALUES (?, ?)", 
                       [(int(x), f"thal_{x}") for x in thal_values])

    
    cursor.execute('''
        CREATE TABLE patients (
            patient_id INTEGER PRIMARY KEY,
            age INTEGER,
            sex INTEGER
        )
    ''')
    patients_data = df[['patient_id', 'age', 'sex']].drop_duplicates()
    patients_data.to_sql('patients', conn, if_exists='append', index=False)
    
    
    cursor.execute('''
        CREATE TABLE exams (
            exam_id INTEGER PRIMARY KEY AUTOINCREMENT,
            patient_id INTEGER,
            cp INTEGER,
            trestbps INTEGER,
            chol INTEGER,
            fbs INTEGER,
            restecg INTEGER,
            thalach INTEGER,
            exang INTEGER,
            oldpeak REAL,
            slope INTEGER,
            ca INTEGER,
            thal INTEGER,
            target INTEGER,
            FOREIGN KEY(patient_id) REFERENCES patients(patient_id),
            FOREIGN KEY(cp) REFERENCES lookup_cp(cp_code),
            FOREIGN KEY(restecg) REFERENCES lookup_restecg(restecg_code),
            FOREIGN KEY(slope) REFERENCES lookup_slope(slope_code),
            FOREIGN KEY(thal) REFERENCES lookup_thal(thal_code)
        )
    ''')
    
    
    exams_cols = ['patient_id', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 
                  'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'target']
    exams_data = df[exams_cols]
    exams_data.to_sql('exams', conn, if_exists='append', index=False)
    
    conn.commit()
    conn.close()
    print(f"Database created at {db_path}")


create_normalized_db(CSV_PATH, DB_PATH)


## 3. Data Loading 


In [ ]:
def load_data_from_db(db_path):
    conn = sqlite3.connect(db_path)
    
    query = '''
        SELECT 
            p.age, p.sex,
            e.cp, e.trestbps, e.chol, e.fbs, e.restecg, 
            e.thalach, e.exang, e.oldpeak, e.slope, e.ca, e.thal, 
            e.target
        FROM exams e
        JOIN patients p ON e.patient_id = p.patient_id
    '''
    
    df = pd.read_sql_query(query, conn)
    conn.close()
    return df

item_df = load_data_from_db(DB_PATH)
print("Data shape:", item_df.shape)
item_df.head()


## 4. EDA & Preprocessing


In [ ]:
# Class Balance
print("Class Balance:")
print(item_df['target'].value_counts(normalize=True))

# Summary Stats
print("Summary Stats:")
print(item_df.describe())


In [ ]:
# Define Preprocessing Pipeline

X = item_df.drop('target', axis=1)
y = item_df['target']

# Categorical columns to encode
cat_features = ['cp', 'restecg', 'slope', 'thal']
# Numeric columns to scale (all others except sex, fbs, exang which are binary but scaling them is fine too usually, or pass through)
# For simplicity, we'll scale all non-categorical features.
num_features = [c for c in X.columns if c not in cat_features]

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), num_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), cat_features)
    ]
)

# Split Data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, stratify=y, random_state=RANDOM_SEED)

print("Train shape:", X_train.shape)
print("Test shape:", X_test.shape)


## 5. Experiment Loop (16 Runs)


In [ ]:
# pip install -q dagshub mlflow

In [ ]:
import dagshub
dagshub.init(repo_owner="emmu9520", repo_name="my-first-repo", mlflow=True)

In [ ]:

models_registry = {
    'LogisticRegression': LogisticRegression(random_state=RANDOM_SEED, max_iter=1000),
    'RandomForest': RandomForestClassifier(random_state=RANDOM_SEED),
    'SVC': SVC(random_state=RANDOM_SEED, probability=True),
    'GradientBoosting': GradientBoostingClassifier(random_state=RANDOM_SEED)
}


def objective(trial, algo_name, X, y):
    if algo_name == 'LogisticRegression':
        C = trial.suggest_loguniform('C', 1e-4, 1e2)
        model = LogisticRegression(C=C, random_state=RANDOM_SEED, max_iter=1000)
    elif algo_name == 'RandomForest':
        n_estimators = trial.suggest_int('n_estimators', 10, 200)
        max_depth = trial.suggest_int('max_depth', 2, 20)
        model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=RANDOM_SEED)
    elif algo_name == 'SVC':
        C = trial.suggest_loguniform('C', 1e-3, 1e2)
        gamma = trial.suggest_loguniform('gamma', 1e-3, 1e2)
        model = SVC(C=C, gamma=gamma, random_state=RANDOM_SEED, probability=True)
    elif algo_name == 'GradientBoosting':
        n_estimators = trial.suggest_int('n_estimators', 10, 200)
        learning_rate = trial.suggest_loguniform('learning_rate', 0.001, 0.1)
        max_depth = trial.suggest_int('max_depth', 2, 10)
        model = GradientBoostingClassifier(n_estimators=n_estimators, learning_rate=learning_rate, max_depth=max_depth, random_state=RANDOM_SEED)
    else:
        return 0
        
    score = cross_val_score(model, X, y, cv=3, scoring='f1').mean()
    return score


optuna.logging.set_verbosity(optuna.logging.WARNING)

experiment_results = []
mlflow_experiment_name = "heart_classification_16_runs"
mlflow.set_experiment(mlflow_experiment_name)


algorithms = ['LogisticRegression', 'RandomForest', 'SVC', 'GradientBoosting']
pca_flags = [False, True]
tuning_flags = [False, True]

run_count = 0

for algo in algorithms:
    for use_pca in pca_flags:
        for use_tuning in tuning_flags:
            run_count += 1
            run_name = f"{algo}_{'PCA' if use_pca else 'NoPCA'}_{'Tuned' if use_tuning else 'Default'}"
            print(f"Running scenario {run_count}/16: {run_name}")
            
            with mlflow.start_run(run_name=run_name):
                
                steps = [('preprocessor', preprocessor)]
                
                if use_pca:
                    
                    steps.append(('pca', PCA(n_components=0.95, random_state=RANDOM_SEED)))
                
                
                prep_pipe = Pipeline(steps)
                X_train_transformed = prep_pipe.fit_transform(X_train, y_train)
                
                
                best_params = {}
                clf = None
                
                if use_tuning:
                    study = optuna.create_study(direction='maximize')
                    study.optimize(lambda trial: objective(trial, algo, X_train_transformed, y_train), n_trials=10)
                    best_params = study.best_params
                    
                    if algo == 'LogisticRegression':
                        clf = LogisticRegression(**best_params, random_state=RANDOM_SEED, max_iter=1000)
                    elif algo == 'RandomForest':
                        clf = RandomForestClassifier(**best_params, random_state=RANDOM_SEED)
                    elif algo == 'SVC':
                        clf = SVC(**best_params, random_state=RANDOM_SEED, probability=True)
                    elif algo == 'GradientBoosting':
                        clf = GradientBoostingClassifier(**best_params, random_state=RANDOM_SEED)
                else:
                    clf = clone(models_registry[algo])
                    best_params = "default"

                
                full_pipeline = Pipeline(steps + [('classifier', clf)])
                
                full_pipeline.fit(X_train, y_train)
                
                
                y_pred = full_pipeline.predict(X_test)
                f1 = f1_score(y_test, y_pred)
                acc = accuracy_score(y_test, y_pred)
                
                print(f"  --> F1: {f1:.4f}")
                
                
                mlflow.log_param("algorithm", algo)
                mlflow.log_param("use_pca", use_pca)
                mlflow.log_param("use_tuning", use_tuning)
                mlflow.log_params(best_params if isinstance(best_params, dict) else {"params": "default"})
                
                mlflow.log_metric("f1_score", f1)
                mlflow.log_metric("accuracy", acc)
                
                
                model_filename = f"model_{run_name}.joblib"
                joblib.dump(full_pipeline, model_filename)
                mlflow.log_artifact(model_filename)
                
               
                metrics = {"f1_score": f1, "accuracy": acc}
                metrics_filename = f"metrics_{run_name}.json"
                with open(metrics_filename, "w") as f:
                    json.dump(metrics, f)
                mlflow.log_artifact(metrics_filename)
                
                
                meta = {
                    "run_name": run_name,
                    "algorithm": algo,
                    "use_pca": use_pca,
                    "use_tuning": use_tuning,
                    "best_params": best_params
                }
                meta_filename = f"metadata_{run_name}.json"
                with open(meta_filename, "w") as f:
                    json.dump(meta, f)
                mlflow.log_artifact(meta_filename)
                
                
                os.remove(model_filename)
                os.remove(metrics_filename)
                os.remove(meta_filename)
                
                
                experiment_results.append({
                    "run_name": run_name,
                    "f1": f1,
                    "model": full_pipeline,
                    "metadata": meta
                })

print("All experiments completed.")


## 6. Best Model Selection


In [ ]:

best_run = max(experiment_results, key=lambda x: x['f1'])

print(f"Best Run: {best_run['run_name']} with F1: {best_run['f1']:.4f}")


best_model_path = os.path.join(MODELS_DIR, "best_model.joblib")
joblib.dump(best_run['model'], best_model_path)


best_meta_path = os.path.join(MODELS_DIR, "best_model_metadata.json")
with open(best_meta_path, "w") as f:
    json.dump(best_run['metadata'], f)

print(f"Saved best model to {best_model_path}")
print(f"Saved best metadata to {best_meta_path}")
